In [6]:
pip install sounddevice

  Obtaining dependency information for sounddevice from https://files.pythonhosted.org/packages/d4/09/bfdd393f1bb1b90b4a6849b84972b7059c95e36818cc489922228d58cc63/sounddevice-0.4.7-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/200.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/200.1 kB ? eta -:--:--
   -- ------------------------------------- 10.2/200.1 kB ? eta -:--:--
   -- ------------------------------------- 10.2/200.1 kB ? eta -:--:--
   -- ------------------------------------- 10.2/200.1 kB ? eta -:--:--
   -- ------------------------------------- 10.2/200.1 kB ? eta -:--:--
   -- ------------------------------------- 10.2/200.1 kB ? eta -:--:--
   -- ------------------------------------- 10.2/200.1 kB ? eta -:--:--
   -- ------------------------------------- 10.2/200.1 kB ? eta -:--:--
   -- ------------------------------------- 10.2/200.1 kB ? eta -:--:--
   -- ------------------------------------- 10.2/200.1 kB ? eta -:--:

In [24]:
pip install pyaudio


  Obtaining dependency information for pyaudio from https://files.pythonhosted.org/packages/82/d8/f043c854aad450a76e476b0cf9cda1956419e1dacf1062eb9df3c0055abe/PyAudio-0.2.14-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/164.1 kB ? eta -:--:--
   -- ------------------------------------- 10.2/164.1 kB ? eta -:--:--
   ------- ------------------------------- 30.7/164.1 kB 435.7 kB/s eta 0:00:01
   -------------------------------------- 164.1/164.1 kB 979.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [27]:
pip install wavio

  Obtaining dependency information for wavio from https://files.pythonhosted.org/packages/87/9a/50c7b7107b98330d295f82a38a8d33210fb33ca472cbdd18dce4ca2fdaec/wavio-0.0.9-py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


In [2]:
import librosa
from scipy.io import wavfile as wav
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime
import sounddevice as sd
import wavio
from tkinter import *
from tkinter import filedialog

# Audio dataset path
audio_dataset_path = r"C:\Users\Rachana Varadaraj\Downloads\UrbanSound8K\UrbanSound8K\audio"
metadata = pd.read_csv('UrbanSound8K.csv')
metadata.head()

# Feature extractor function
def features_extractor(file):
    audio, sample_rate = librosa.load(file, res_type='kaiser_fast') 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)
    return mfccs_scaled_features

# Extract features from all audio files
extracted_features = []
for index_num, row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path), 'fold' + str(row["fold"]), str(row["slice_file_name"]))
    final_class_labels = row["class"]
    data = features_extractor(file_name)
    extracted_features.append([data, final_class_labels])

extracted_features_df = pd.DataFrame(extracted_features, columns=['feature', 'class'])
extracted_features_df.head()

# Split the dataset into independent and dependent datasets
X = np.array(extracted_features_df['feature'].tolist())
y = np.array(extracted_features_df['class'].tolist())

# Label encoding
labelencoder = LabelEncoder()
y = to_categorical(labelencoder.fit_transform(y))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Build the model
model = Sequential()
model.add(Dense(100, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1]))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')


3555it [11:06,  6.12it/s]C:\Users\Rachana Varadaraj\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1323
  warnings.warn(
8325it [28:29,  7.08it/s]C:\Users\Rachana Varadaraj\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
8328it [28:29, 11.48it/s]C:\Users\Rachana Varadaraj\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1523
  warnings.warn(
8732it [29:35,  4.92it/s]


In [3]:

# Train the model
num_epochs = 100
num_batch_size = 32
checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.keras', verbose=1, save_best_only=True)
start = datetime.now()
model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)
duration = datetime.now() - start
print("Training completed in time: ", duration)

# Evaluate the model
test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(test_accuracy[1])


Epoch 1/100


214/219 [============================>.] - ETA: 0s - loss: 11.8851 - accuracy: 0.1253
Epoch 1: val_loss improved from inf to 2.28441, saving model to saved_models\audio_classification.keras
219/219 [==============================] - 7s 14ms/step - loss: 11.7204 - accuracy: 0.1251 - val_loss: 2.2844 - val_accuracy: 0.1196
Epoch 2/100
218/219 [============================>.] - ETA: 0s - loss: 2.6042 - accuracy: 0.1272
Epoch 2: val_loss improved from 2.28441 to 2.27315, saving model to saved_models\audio_classification.keras
219/219 [==============================] - 3s 13ms/step - loss: 2.6040 - accuracy: 0.1273 - val_loss: 2.2731 - val_accuracy: 0.1231
Epoch 3/100
217/219 [============================>.] - ETA: 0s - loss: 2.3332 - accuracy: 0.1322
Epoch 3: val_loss improved from 2.27315 to 2.17923, saving model to saved_models\audio_classification.keras
219/219 [==============================] - 2s 11ms/step - loss: 2.3321 - accuracy: 0.1321 - val_loss: 2.1792 - val_accura

In [10]:

# Create the UI
root = Tk()
root.title("Audio Classification")
root.geometry("500x300")

def browse_file():
    filename = filedialog.askopenfilename(filetypes=[("WAV files", "*.wav")])
    if filename:
        predict_class(filename)

def predict_class(file_path):
    audio, sample_rate = librosa.load(file_path, res_type='kaiser_fast')
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)
    mfccs_scaled_features = mfccs_scaled_features.reshape(1, -1)
    probabilities = model.predict(mfccs_scaled_features)
    predicted_class = np.argmax(probabilities, axis=1)
    prediction_class = labelencoder.inverse_transform(predicted_class)
    result_label.config(text=f"Predicted Class: {prediction_class[0]}")

def record_audio():
    fs = 44100  # Sample rate
    seconds = 3  # Duration of recording
    print("Recording...")
    audio = sd.rec(int(seconds * fs), samplerate=fs, channels=2)
    sd.wait()  # Wait until recording is finished
    print("Recording completed")
    wavio.write("recorded_audio.wav", audio, fs, sampwidth=2)
    predict_class("recorded_audio.wav")

browse_button = Button(root, text="Browse", font=("Helvetica", 16), command=browse_file)
browse_button.pack(pady=20)

record_button = Button(root, text="Record", font=("Helvetica", 16), command=record_audio)
record_button.pack(pady=20)

result_label = Label(root, text="Predicted Class: ", font=("Helvetica", 16))
result_label.pack(pady=20)


root.mainloop()

Recording...
Recording completed
1/1 [==============================] - 0s 38ms/step
